# Evaluate a crypto currency project

## Pip install and Imports

In [1]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pycoingecko
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org ratelimit
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org tqdm

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

## Get all the coins and their corresponding ids

In [3]:
MAX_MARKET_CAP = 100_000_000 # target only small market cap crypto currencies
MIN_MARKET_CAP = 50_000
MIN_VOLUME = 50_000
USED_FIELDS = set(["id", "symbol", "market_cap"]) # to avoid having too much data
MAX_API_CALLS = 10
MAX_API_CALLS_PERIOD = 60 # 1 minute
PRICE_CHANGE_PERCENTAGE_PERIODS = '7d,14d,30d,200d'
PERCENTAGE_CHANGE = 100

from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=MAX_API_CALLS, period=MAX_API_CALLS_PERIOD)
def get_coins_markets(*args, **kwargs) -> list:
    return cg.get_coins_markets(*args, **kwargs)

max_market_cap_reached = False
map_id_to_symbol = {}
all_coins_ids = []
valid_markets = []
page = 1

while True:
    print(f"page = {page}")
    markets = get_coins_markets(vs_currency="usd", per_page=250, page=page, order="market_cap_asc", price_change_percentage=PRICE_CHANGE_PERCENTAGE_PERIODS)
    if not markets:
        break
    #new_markets = [
    #    market
    #    for market in markets if market["market_cap"] is not None and
    #    MIN_MARKET_CAP <= market["market_cap"] <= MAX_MARKET_CAP and (
    #        market["price_change_percentage_7d_in_currency"] is not None and market["price_change_percentage_7d_in_currency"] > PERCENTAGE_CHANGE# or
            #market["price_change_percentage_14d_in_currency"] is not None and market["price_change_percentage_14d_in_currency"] > PERCENTAGE_CHANGE or
            #market["price_change_percentage_30d_in_currency"] is not None and market["price_change_percentage_30d_in_currency"] > PERCENTAGE_CHANGE or
            #market["price_change_percentage_200d_in_currency"] is not None and market["price_change_percentage_200d_in_currency"] > PERCENTAGE_CHANGE
    #    ) and market["total_volume"] > MIN_VOLUME
    #]
    valid_markets += markets #new_markets
    page += 1
map_id_to_symbol = {market["id"]: market["symbol"] for market in valid_markets}
all_coins_ids = list(map_id_to_symbol.keys())

page = 1
page = 2
page = 3
page = 4
page = 5
page = 6
page = 7
page = 8
page = 9
page = 10
page = 11
page = 12
page = 13
page = 14
page = 15
page = 16
page = 17
page = 18
page = 19
page = 20
page = 21
page = 22
page = 23
page = 24
page = 25
page = 26
page = 27
page = 28
page = 29
page = 30
page = 31
page = 32
page = 33
page = 34
page = 35
page = 36
page = 37
page = 38
page = 39
page = 40
page = 41
page = 42
page = 43
page = 44
page = 45
page = 46
page = 47
page = 48
page = 49
page = 50
page = 51
page = 52
page = 53
page = 54
page = 55


In [ ]:
from tqdm import tqdm

@sleep_and_retry
@limits(calls=MAX_API_CALLS, period=MAX_API_CALLS_PERIOD)
def get_coin_by_id(*args, **kwargs) -> list:
    return cg.get_coin_by_id(*args, **kwargs)

#coins_data = []
for i in tqdm(range(1532, len(valid_markets))):
    market = valid_markets[i]
    try:
        coins_data.append(
            cg.get_coin_by_id(market["id"], localization=False, market_data=False)
        )
    except ValueError as ve:
        print(str(ve))
        if str(ve) != "{'error': 'Could not find coin with the given id'}":
            print(str(ve))
            raise ve

 32%|███▏      | 3708/11743 [18:53:22<1:09:15,  1.93it/s]       

{'error': 'Could not find coin with the given id'}


 35%|███▌      | 4158/11743 [19:23:44<52:24,  2.41it/s]   

{'error': 'Could not find coin with the given id'}


 37%|███▋      | 4297/11743 [19:32:30<39:23,  3.15it/s]   

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(coins_data)

In [ ]:
df.info()

In [ ]:
import requests
import json

#its bad practice to place your bearer token directly into the script (this is just done for illustration purposes)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKSKigEAAAAAv%2BSQBzUZexoYCs3xep2MNCzeAbo%3DMHoffAoGn730RtKwg8XECvKKV0Sjid0a6KWtKuwqUqVit0jdAs"

#define search twitter function
def search_twitter(query: str, tweet_fields: str, end_time: str, bearer_token = BEARER_TOKEN): 
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    print(headers)

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&tweet.fields={}&end_time={}".format(
        query, tweet_fields, end_time
    )
    print(url)
    response = requests.request("GET", url, headers=headers, verify=False)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
from datetime import datetime, timedelta
now = datetime.now()

for market in valid_markets:
    coin_id = market["id"]
    print(f"coin_id = {coin_id}")
    coin_symbol = market["symbol"]
    print(f"coin_symbol = {coin_symbol}")
    end_time = now - timedelta(days=7)
    query=f"\${coin_symbol}"
    response = search_twitter(query=f"\${coin_symbol}", tweet_fields="text,author_id,created_at", end_time=end_time.isoformat()[:21] + "Z", bearer_token=BEARER_TOKEN)
    print(response)
    break

In [ ]:
def get_min_days_percentage_change(market: dict) -> int:
    periods = [7, 14, 30, 200]
    last = None
    for period in periods:
        if market[f"price_change_percentage_{period}d_in_currency"] is not None:
            last = period
    return last

enhanced_valid_markets = [
    market | {"min_days_percentage_change": get_min_days_percentage_change(market)}
    for market in valid_markets
]